In [8]:
pip install langchain==0.0.208 deeplake openai tiktoken


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from langchain.llms import OpenAI

In [10]:
# import dotenv file
!pip install python-dotenv

from dotenv import load_dotenv
load_dotenv('.env')


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


True

In [11]:
llm = OpenAI(model="text-davinci-003", temperature=0.9)

In [12]:
# testing llm

text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurnce and prefers outdoor activities."
print(llm(text))



1. Long-Distance Jogging: Begin by jogging for 45 minutes every other day, increasing time every week until you reach 1.5 hours. Increase intensity by running at a faster pace for 1 mile every other day.

2. Interval Training: Alternating between sprinting and jogging is great for boosting cardio endurance and burning calories. Try sprinting for 30 seconds then jogging for 1 minute for a total of 20 minutes. Increase intensity by sprinting for 45 seconds and jogging for 45 seconds or 1 minute sprints and 30 second jogs.

3. Hill Runs: Hill runs can be great for improving leg strength and cardiovascular endurance. Find a hill that takes you at least 3 minutes to run up and try running up it as fast as you can. As you get stronger, increase the length of the hill.

4. Outdoor Circuit Training: Create a circuit with your favorite outdoor activities such as running, jumping rope, push-ups, plyometrics, squats, burpees, and medicine ball exercises. Alternate your exercises for 30 seconds 

In [13]:
# Using the prompt template with llm wrapper

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variables

print(chain.run("eco-friendly water bottles"))



Eco Aqua Bottles.


In [14]:
# from langchain.llm import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input='What can you do?')
conversation.predict(input='How can you help me with data analysis?')

# Display the conversation
print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI:  Hi there! I'm an AI created to help people with their daily tasks. I'm programmed to understand natural language and respond to questions and commands. I'm also able to learn from my interactions with people, so I'm constantly growing and improving. I'm excited to help you out!
Human: What can yo

In [23]:
# import ACTIVELOOP_TOKEN key from .env file
import os
os.environ["ACTIVELOOP_TOKEN"] = "YOUR_ACTIVELOOP_TOKEN"

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(model='text-davinci-003', temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# creation of the document
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# creating Deep Lake Dataset
my_activeloop_org_id = "nikhilsharma26500"
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


|

Dataset(path='hub://nikhilsharma26500/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['9383256b-1735-11ee-b60b-80b6558dba70',
 '9383256c-1735-11ee-98a7-80b6558dba70']

In [27]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [28]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions about historical figures.",
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbone=True
)

In [29]:
response = agent.run("When was Napoleon Bonaparte born?")
print(response)

Napoleon Bonaparte was born on 15 August 1769.
